# Task 5
# BERT Modules

## 5.1 Base BERT

### 5.1.1 Import Packages & Setup
First, I installed the  TensorFlow library to enable building and training deep learning models.

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from transformers import BertTokenizerFast, BertPreTrainedModel, BertModel
from transformers import AutoTokenizer, AutoModel
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AutoConfig

# Read in three datasets
data_train = pd.read_csv('data_train.csv')
data_val = pd.read_csv('data_val.csv')
data_test = pd.read_csv('data_test.csv')

# For build
# data_train = data_train.sample(frac=0.003, random_state=42).reset_index(drop=True)
# data_val = data_val.sample(frac=0.05, random_state=42).reset_index(drop=True)
# data_test = data_test.sample(frac=0.01, random_state=42).reset_index(drop=True)

full_cols = [
    'sdoh_community_present', 'sdoh_community_absent', 'sdoh_education',
    'sdoh_economics', 'sdoh_environment', 'behavior_alcohol',
    'behavior_tobacco', 'behavior_drug'
]

#### Convert to Hugging Face Dataset
Using the Dataset makes handling data is easier to use Hugging Face training tools

In [2]:
# Convert Pandas DataFrames to Hugging Face Dataset
train_ds = Dataset.from_pandas(data_train)
val_ds = Dataset.from_pandas(data_val)
test_ds = Dataset.from_pandas(data_test)

#### Preprocess the Dataset

In [3]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# 2. tokenizer
modelname = 'bert-base-uncased'
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


def tokenize(batch):
    return tokenizer(batch['social_history'], padding='max_length',truncation=True,max_length=128)
    
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

def combine_labels(batch):
    labels = []
    for i in range(len(batch[full_cols[0]])):
        row = []
        for cols in full_cols:
            row.append(batch[cols][i])
        labels.append(row)
    batch['labels'] = labels
    return batch

train_ds = train_ds.map(combine_labels, batched=True)
val_ds = val_ds.map(combine_labels, batched=True)
test_ds = test_ds.map(combine_labels, batched=True)


# Remove extra columns to avoid passing extra columns to model
use_cols=['input_ids', 'attention_mask', 'labels']
train_ds = train_ds.remove_columns([cols for cols in train_ds.column_names if cols not in use_cols])
val_ds = val_ds.remove_columns([cols for cols in val_ds.column_names if cols not in use_cols])
test_ds = test_ds.remove_columns([cols for cols in test_ds.column_names if cols not in use_cols])

Map:   0%|          | 0/4917 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

Map:   0%|          | 0/4917 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

Define a simple tokenisation function that takes a batch of text and labels as input, takes out the text part of it, and returns the tokenised text and the labels.

### 5.1.2 Create the Standard BERT Module

Similarly, I build a model that shares a single encoder and uses a separate classification head (nn.Linear) for each task
- Merges logits from all tasks into a unified tensor with padding for alignment
- Supports joint loss computation across tasks using CrossEntropyLoss

In [4]:
num_labels_list=[2,2,2,3,3,5,5,5]

# Use the pretrained BertPreTrainedModel class for initialization**
class MultiTaskBertModel(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels_list = num_labels_list
        self.num_tasks = 8
        
        #self.bert = AutoModel.from_config(config)
        self.bert = AutoModel.from_pretrained('bert-base-uncased', config=config)

        # Add a Hidden layer before classification heads
        #self.hidden = nn.Sequential(
            #nn.Linear(config.hidden_size, 256),
            #nn.GELU(),
            # Add a 50% dropout
            #nn.Dropout(0.5)
        #)
        
        # Use one classification head per task and register them using nn.ModuleList.
        self.classifiers = nn.ModuleList([nn.Linear(config.hidden_size, num_labels) for num_labels in num_labels_list])
        #self.classifiers = nn.ModuleList([nn.Linear(256, num_labels) for num_labels in num_labels_list])
        self.init_weights()


        
    # Define forward pass for multi-task model to compute combined loss for joint training
    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        # hidden_output = self.hidden(pooled_output)
        logits_list = [classifier(pooled_output) for classifier in self.classifiers]

        batch_size = pooled_output.size(0)
        device = pooled_output.device

        # As Using Trainer needs unified logits with consistent shape across tasks. Combine logits from 8 tasks and use -1e9 as padding to mask unused positions.
        # maximun of num_labels is 5

        # Initialize logits with zeros, shape: (batch_size, num_tasks, max_labels)
        logits = torch.zeros(batch_size, self.num_tasks, 5, device=device)
        #logits = torch.full((batch_size, self.num_tasks, 5), -1e9, device=device)
        for i, logit_task in enumerate(logits_list):

            num_labels = self.num_labels_list[i]
            #print(f"Task {i} logits shape: {logit_task.shape}")
            logits[:, i, :num_labels] = logit_task

        loss_fct = nn.CrossEntropyLoss()
        loss = 0
        for i in range(self.num_tasks):
            loss += loss_fct(logits[:, i, :self.num_labels_list[i]], labels[:, i])

        return (loss, logits)

config = AutoConfig.from_pretrained('bert-base-uncased')
model = MultiTaskBertModel.from_pretrained('bert-base-uncased', config=config)

Some weights of MultiTaskBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifiers.0.bias', 'classifiers.0.weight', 'classifiers.1.bias', 'classifiers.1.weight', 'classifiers.2.bias', 'classifiers.2.weight', 'classifiers.3.bias', 'classifiers.3.weight', 'classifiers.4.bias', 'classifiers.4.weight', 'classifiers.5.bias', 'classifiers.5.weight', 'classifiers.6.bias', 'classifiers.6.weight', 'classifiers.7.bias', 'classifiers.7.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 5.1.3 Set Up Evaluation

Same as above, macro and weighted scores are chosen to evaluate model's performance

In [5]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.astype(int)

    precision_macro, recall_macro, f1_macro = [], [], []
    precision_weighted, recall_weighted, f1_weighted = [], [], []
    acc_list = []

    # 用于保存每个任务的 Macro F1 分数
    task_f1_scores = {}

    for i in range(8):
        logit_task = logits[:, i, :num_labels_list[i]]
        preds_i = np.argmax(logit_task, axis=1)
        y_true = labels[:, i]

        acc = accuracy_score(y_true, preds_i)
        acc_list.append(acc)

        # macro
        prec = precision_score(y_true, preds_i, average='macro', zero_division=0)
        rec = recall_score(y_true, preds_i, average='macro', zero_division=0)
        f1 = f1_score(y_true, preds_i, average='macro', zero_division=0)
        precision_macro.append(prec)
        recall_macro.append(rec)
        f1_macro.append(f1)
        

        # 保存每个任务的 Macro F1 score，使用 full_cols[i] 作为任务的名称
        task_f1_scores[f'{full_cols[i]}_macro_f1'] = f1  # 使用任务名称作为键

        # weighted
        prec_w = precision_score(y_true, preds_i, average='weighted', zero_division=0)
        rec_w = recall_score(y_true, preds_i, average='weighted', zero_division=0)
        f1_w = f1_score(y_true, preds_i, average='weighted', zero_division=0)
        precision_weighted.append(prec_w)
        recall_weighted.append(rec_w)
        f1_weighted.append(f1_w)

    metrics = {
        'macro_acc': np.mean(acc_list),

        'macro_precision': np.mean(precision_macro),
        'macro_recall': np.mean(recall_macro),
        'macro_f1': np.mean(f1_macro),

        'weighted_precision': np.mean(precision_weighted),
        'weighted_recall': np.mean(recall_weighted),
        'weighted_f1': np.mean(f1_weighted),
    }

    metrics.update(task_f1_scores)

    return metrics

### 5.1.4 Set up Training

In [6]:
# Set up a few arguments in a seperate TrainingArguments class
training_args = TrainingArguments(
    output_dir='./Final_Project',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_strategy='steps',    
    logging_steps=50,
    push_to_hub=False,
    #report_to="tensorboard",
    #Evaluate after each epoch and save checkpoints to easily resume training, add epochs
    eval_strategy='epoch',
    save_strategy='epoch',

    # Set to automatically load the the best macro_f1 score model on validation set
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    # tokenizer=tokenizer,
    processing_class= tokenizer,
    compute_metrics=compute_metrics,
)

### 5.1.5 Train the Module & Test

In [45]:
trainer.train()

test_metrics = trainer.evaluate(test_ds)
print("Best model Performance on test dataset: ")
print(test_metrics)

Epoch,Training Loss,Validation Loss,Macro Acc,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Sdoh Community Present Macro F1,Sdoh Community Absent Macro F1,Sdoh Education Macro F1,Sdoh Economics Macro F1,Sdoh Environment Macro F1,Behavior Alcohol Macro F1,Behavior Tobacco Macro F1,Behavior Drug Macro F1
1,3.910400,3.701662,0.848934,0.655736,0.578165,0.566819,0.828689,0.848934,0.823307,0.910095,0.557343,0.492300,0.550012,0.607533,0.475393,0.562691,0.379181
2,2.514600,2.322437,0.916825,0.806499,0.718903,0.740230,0.910222,0.916825,0.909670,0.973265,0.876270,0.647328,0.838419,0.632662,0.670736,0.778025,0.505133
3,1.664400,1.758617,0.936611,0.852275,0.786118,0.804176,0.933339,0.936611,0.932459,0.979435,0.944930,0.828956,0.892471,0.639514,0.770375,0.826875,0.550853
4,1.328400,1.532853,0.942417,0.887173,0.815248,0.832272,0.940539,0.942417,0.940114,0.977352,0.950455,0.864768,0.886598,0.704612,0.821946,0.852799,0.599649
5,0.953300,1.396813,0.947986,0.897378,0.838233,0.854001,0.946808,0.947986,0.946342,0.981575,0.946323,0.903317,0.890066,0.757534,0.842351,0.877696,0.633145
6,0.782600,1.305240,0.950474,0.897880,0.842770,0.856179,0.949265,0.950474,0.949020,0.975350,0.949719,0.893650,0.902314,0.758244,0.846654,0.887983,0.635515
7,0.561000,1.296835,0.950948,0.893783,0.846942,0.860911,0.949855,0.950948,0.949735,0.977455,0.949719,0.877288,0.893195,0.801522,0.862729,0.886555,0.638828
8,0.498600,1.266390,0.954028,0.913661,0.858461,0.875488,0.953555,0.954028,0.953008,0.979482,0.956741,0.904098,0.912758,0.798439,0.875257,0.881556,0.695573
9,0.361000,1.247788,0.955687,0.910708,0.874528,0.886149,0.955211,0.955687,0.954997,0.978419,0.952641,0.906155,0.903038,0.839612,0.870337,0.897282,0.741712
10,0.320200,1.236513,0.954621,0.910801,0.881029,0.892894,0.954464,0.954621,0.954207,0.975378,0.952641,0.893650,0.905183,0.898923,0.865713,0.897208,0.754459


C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\en

Best model Performance on test dataset: 
{'eval_loss': 1.2566423416137695, 'eval_macro_acc': 0.9586894586894587, 'eval_macro_precision': 0.9098005480315166, 'eval_macro_recall': 0.8891243667621189, 'eval_macro_f1': 0.8976103070831851, 'eval_weighted_precision': 0.9584484113093603, 'eval_weighted_recall': 0.9586894586894587, 'eval_weighted_f1': 0.9583890595215988, 'eval_sdoh_community_present_macro_f1': 0.9824086949258307, 'eval_sdoh_community_absent_macro_f1': 0.941429498470381, 'eval_sdoh_education_macro_f1': 0.8968152866242038, 'eval_sdoh_economics_macro_f1': 0.9124918716761746, 'eval_sdoh_environment_macro_f1': 0.9162276768414875, 'eval_behavior_alcohol_macro_f1': 0.8556693389993264, 'eval_behavior_tobacco_macro_f1': 0.9245133722814941, 'eval_behavior_drug_macro_f1': 0.7513267168465827, 'eval_runtime': 54.9432, 'eval_samples_per_second': 19.165, 'eval_steps_per_second': 1.201, 'epoch': 15.0}


### 5.2 Bio-Clinical BERT Model

In [7]:
# Choose Bio-ClinicalBERT as model 2
modelname2 = 'emilyalsentzer/Bio_ClinicalBERT'
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')

##### Convert these three datasets to Hugging Face Dataset again

In [8]:
train_ds = Dataset.from_pandas(data_train)
val_ds = Dataset.from_pandas(data_val)
test_ds = Dataset.from_pandas(data_test)

### 5.2.1 Preprocess

In [9]:
def tokenize(batch):
    return tokenizer(batch['social_history'], padding='max_length',truncation=True,max_length=128)
    
train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)
test_ds = test_ds.map(tokenize, batched=True)

def combine_labels(batch):
    labels = []
    for i in range(len(batch[full_cols[0]])):
        row = []
        for cols in full_cols:
            row.append(batch[cols][i])
        labels.append(row)
    batch['labels'] = labels
    return batch

train_ds = train_ds.map(combine_labels, batched=True)
val_ds = val_ds.map(combine_labels, batched=True)
test_ds = test_ds.map(combine_labels, batched=True)


# Remove extra columns to avoid passing extra columns to model
use_cols=['input_ids', 'attention_mask', 'labels']
train_ds = train_ds.remove_columns([cols for cols in train_ds.column_names if cols not in use_cols])
val_ds = val_ds.remove_columns([cols for cols in val_ds.column_names if cols not in use_cols])
test_ds = test_ds.remove_columns([cols for cols in test_ds.column_names if cols not in use_cols])

Map:   0%|          | 0/4917 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

Map:   0%|          | 0/4917 [00:00<?, ? examples/s]

Map:   0%|          | 0/1055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1053 [00:00<?, ? examples/s]

### 5.2.2 Create the BioClinicalBERT Module

In [10]:
class MultiTaskBertModel2(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_tasks = 8
        # self.bert = BertModel(config) 
        self.bert = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', config=config)  # Use Bio+Clinical BERT from huggingface
        
        self.classifiers = nn.ModuleList([nn.Linear(config.hidden_size, num_labels) for num_labels in num_labels_list])
        self.num_labels_list = num_labels_list
        self.init_weights()

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output

        logits_list = [classifier(pooled_output) for classifier in self.classifiers]

        batch_size = pooled_output.size(0)
        device = pooled_output.device

        # Initialize logits with zeros, shape: (batch_size, num_tasks, max_labels)
        logits = torch.zeros(batch_size, self.num_tasks, 5, device=device)
        # logits = torch.full((batch_size, self.num_tasks,5), -1e9, device=device)
        for i, logit_task in enumerate(logits_list):
            num_labels = self.num_labels_list[i]
            logits[:, i, :num_labels] = logit_task

        loss_fct = nn.CrossEntropyLoss()
        loss = 0
        for i in range(self.num_tasks):
            loss += loss_fct(logits[:, i, :self.num_labels_list[i]], labels[:, i])

        return (loss, logits)

config = AutoConfig.from_pretrained(modelname2)
model = MultiTaskBertModel2.from_pretrained(modelname2, config=config)

Some weights of MultiTaskBertModel2 were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifiers.0.bias', 'classifiers.0.weight', 'classifiers.1.bias', 'classifiers.1.weight', 'classifiers.2.bias', 'classifiers.2.weight', 'classifiers.3.bias', 'classifiers.3.weight', 'classifiers.4.bias', 'classifiers.4.weight', 'classifiers.5.bias', 'classifiers.5.weight', 'classifiers.6.bias', 'classifiers.6.weight', 'classifiers.7.bias', 'classifiers.7.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 5.2.3Set up training

In [ ]:
默认adam优化器

In [11]:
# Set up a few arguments in a seperate TrainingArguments class
training_args = TrainingArguments(
    output_dir='./Final_Project2',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    logging_strategy='steps',    
    logging_steps=50,
    push_to_hub=False,
    #report_to="tensorboard",
    #Evaluate after each epoch and save checkpoints to easily resume training, add epochs
    eval_strategy='epoch',
    save_strategy='epoch',

    # Set to automatically load the the best macro_f1 score model on validation set
    load_best_model_at_end=True,
    metric_for_best_model='macro_f1',
    greater_is_better=True,
)

trainer2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    # tokenizer=tokenizer,
    processing_class= tokenizer
)

#### Train & Test

In [12]:
trainer2.train()

test_metrics = trainer2.evaluate(test_ds)
print("Performance on test dataset: ")
print(test_metrics)

C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Macro Acc,Macro Precision,Macro Recall,Macro F1,Weighted Precision,Weighted Recall,Weighted F1,Sdoh Community Present Macro F1,Sdoh Community Absent Macro F1,Sdoh Education Macro F1,Sdoh Economics Macro F1,Sdoh Environment Macro F1,Behavior Alcohol Macro F1,Behavior Tobacco Macro F1,Behavior Drug Macro F1
1,3.733800,3.480148,0.863863,0.583767,0.588099,0.576620,0.822185,0.863863,0.838668,0.885264,0.470647,0.492300,0.695792,0.613403,0.477653,0.603042,0.374857
2,2.524200,2.347675,0.913744,0.830754,0.682979,0.706172,0.909429,0.913744,0.901270,0.968105,0.716197,0.651621,0.846188,0.642113,0.599974,0.757656,0.467522
3,1.677500,1.763966,0.934953,0.853512,0.760982,0.787933,0.930510,0.934953,0.929175,0.970094,0.911619,0.777764,0.896979,0.643527,0.684037,0.846764,0.572679
4,1.388700,1.480179,0.945142,0.860823,0.812089,0.830907,0.942533,0.945142,0.942727,0.981533,0.936312,0.841318,0.891117,0.643623,0.806188,0.868439,0.678723
5,0.942500,1.373281,0.949526,0.882330,0.845580,0.858694,0.948452,0.949526,0.948355,0.982509,0.944517,0.883679,0.906433,0.742974,0.831188,0.869776,0.708479
6,0.803500,1.277832,0.952725,0.912232,0.838079,0.855107,0.951953,0.952725,0.951129,0.977299,0.947128,0.846737,0.909935,0.709488,0.860093,0.882454,0.707727
7,0.582100,1.227464,0.953910,0.907465,0.853427,0.867909,0.953077,0.953910,0.952794,0.982530,0.949342,0.893650,0.904754,0.759702,0.854005,0.879714,0.719575
8,0.534100,1.235442,0.953791,0.916702,0.846153,0.862540,0.953465,0.953791,0.952585,0.977352,0.958680,0.889936,0.900669,0.706680,0.865426,0.887577,0.714001
9,0.388600,1.225310,0.956398,0.924391,0.865781,0.885331,0.955921,0.956398,0.955541,0.976253,0.954166,0.911356,0.912837,0.802855,0.863520,0.896223,0.765433
10,0.348700,1.206875,0.956872,0.919542,0.865574,0.883303,0.956306,0.956872,0.956113,0.979458,0.956741,0.893628,0.908801,0.802212,0.861524,0.899059,0.765003


C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\envs\transformer\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\k24034365\.conda\en

Performance on test dataset: 
{'eval_loss': 1.252211570739746, 'eval_macro_acc': 0.9560778727445394, 'eval_macro_precision': 0.9073933692731901, 'eval_macro_recall': 0.8607660058002333, 'eval_macro_f1': 0.8727981196506881, 'eval_weighted_precision': 0.9554850126387042, 'eval_weighted_recall': 0.9560778727445394, 'eval_weighted_f1': 0.955216154636358, 'eval_sdoh_community_present_macro_f1': 0.9698462243349274, 'eval_sdoh_community_absent_macro_f1': 0.9267868730879762, 'eval_sdoh_education_macro_f1': 0.9033117042115573, 'eval_sdoh_economics_macro_f1': 0.9144219159207383, 'eval_sdoh_environment_macro_f1': 0.7741762648353204, 'eval_behavior_alcohol_macro_f1': 0.870937793832842, 'eval_behavior_tobacco_macro_f1': 0.9096333577625522, 'eval_behavior_drug_macro_f1': 0.713270823219591, 'eval_runtime': 73.8317, 'eval_samples_per_second': 14.262, 'eval_steps_per_second': 0.894, 'epoch': 15.0}
